
# Case Telecom 

#### Criado por: Caio Araujo - 18 de Agosto de 2025




## 1) Importação e configuração



In [0]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score,
    confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay, brier_score_loss
)
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")


## 2) Importação dos dados



In [0]:
#Carregamento dos dados
df_nps = pd.read_csv('/Workspace/Users/caio.edaraujo@gmail.com/Case_Bradesco/customer_nps.csv', sep=',')
df_social = pd.read_csv('/Workspace/Users/caio.edaraujo@gmail.com/Case_Bradesco/customer_social.csv', sep=',')
df_original = pd.read_csv('/Workspace/Users/caio.edaraujo@gmail.com/Case_Bradesco/customer_original.csv', sep=',')

In [0]:
#Visualização do df_nps
df_nps.head(10)

,cpf,NPS
0,10433218100,9
1,96001338914,9
2,8386379499,9
3,2654235114,8
4,16155940789,9
5,81618495950,9
6,31034131656,9
7,47525534144,9
8,92832764851,10
9,35030564160,7


In [0]:
#Visualização do df_social
df_social.head(10)

,cpf,gender,SeniorCitizen,Partner,Dependents
0,10433218100,Female,0,Yes,Yes
1,96001338914,Female,0,No,No
2,8386379499,Female,0,Yes,No
3,2654235114,Female,0,No,No
4,16155940789,Male,0,No,No
5,81618495950,Male,0,No,No
6,31034131656,Male,0,Yes,No
7,47525534144,Female,0,Yes,Yes
8,92832764851,Female,0,Yes,Yes
9,35030564160,Female,1,Yes,No


In [0]:
#Visualização do df_original
df_original.head(10)

,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,cpf
0,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No,104.332.181-00
1,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No,960.013.389-14
2,68,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Mailed check,118.60,7990.05,No,083.863.794-99
3,61,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Electronic check,118.60,7365.70,No,026.542.351-14
4,67,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),118.35,7804.15,Yes,161.559.407-89
5,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.20,8547.15,No,816.184.959-50
6,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),117.80,8684.80,Yes,310.341.316-56
7,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),117.60,8308.90,No,475.255.341-44
8,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),117.50,8670.10,No,928.327.648-51
9,48,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Month-to-month,No,Electronic check,117.45,5438.90,Yes,350.305.641-60


In [0]:
#Validando tamanho, formato e quantidade de dados nulos na tabela df_nps.
df_nps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7234 entries, 0 to 7233
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   cpf     7234 non-null   int64
 1   NPS     7234 non-null   int64
dtypes: int64(2)
memory usage: 113.2 KB


In [0]:
#Validando tamanho, formato e quantidade de dados nulos na tabela df_social.
df_social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7234 entries, 0 to 7233
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cpf            7234 non-null   int64 
 1   gender         7234 non-null   object
 2   SeniorCitizen  7234 non-null   int64 
 3   Partner        7234 non-null   object
 4   Dependents     7234 non-null   object
dtypes: int64(2), object(3)
memory usage: 282.7+ KB


In [0]:
#Validando tamanho, formato e quantidade de dados nulos na tabela df_original.
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7234 entries, 0 to 7233
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tenure            7234 non-null   int64  
 1   PhoneService      7234 non-null   object 
 2   MultipleLines     7234 non-null   object 
 3   InternetService   7234 non-null   object 
 4   OnlineSecurity    7234 non-null   object 
 5   OnlineBackup      7234 non-null   object 
 6   DeviceProtection  7234 non-null   object 
 7   TechSupport       7234 non-null   object 
 8   StreamingTV       7234 non-null   object 
 9   StreamingMovies   7234 non-null   object 
 10  Contract          7234 non-null   object 
 11  PaperlessBilling  7234 non-null   object 
 12  PaymentMethod     7234 non-null   object 
 13  MonthlyCharges    7234 non-null   float64
 14  TotalCharges      7234 non-null   float64
 15  Churn             7234 non-null   object 
 16  cpf               7234 non-null   object 


In [0]:
#Ajuste no formato do campo cpf do df_original, afim de padronizar com os demais.
df_original['cpf'] = df_original['cpf'].str.replace('.', '', regex=False).str.replace('-', '', regex=False)

#Ajuste no formato do campo cpf para inteiro.
df_original['cpf'] = df_original['cpf'].astype('int64')

In [0]:
#Realizando a união dos três df utilizando o cpf como chave primária.
df_merged = df_original.merge(df_nps[['cpf', 'NPS']], on='cpf', how='left')
df_final = df_merged.merge(df_social, on='cpf', how='left')

df_final.head(10)

,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,cpf,NPS,gender,SeniorCitizen,Partner,Dependents
0,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No,10433218100,9,Female,0,Yes,Yes
1,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No,10433218100,9,Female,0,Yes,Yes
2,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No,10433218100,9,Female,0,Yes,Yes
3,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No,10433218100,9,Female,0,Yes,Yes
4,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No,96001338914,9,Female,0,No,No
5,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No,96001338914,9,Female,0,No,No
6,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No,96001338914,9,Female,0,No,No
7,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No,96001338914,9,Female,0,No,No
8,68,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Mailed check,118.60,7990.05,No,8386379499,9,Female,0,Yes,No
9,68,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Mailed check,118.60,7990.05,No,8386379499,9,Female,0,Yes,No



## 3) Limpeza, validação e formatação


In [0]:
#Criando uma cópia do df_final para a realização dos estudos.
df = df_final.copy()

# Conversões numéricas
for col in ["MonthlyCharges", "TotalCharges"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Deduplicar por CPF
id_col = "cpf" if "cpf" in df.columns else None
if id_col is not None:
    before = len(df)
    df = df.drop_duplicates(subset=[id_col]).reset_index(drop=True)
    print(f"Deduplicação por {id_col}: {before} -> {len(df)}")

# Remover linhas sem Churn ou NPS
df = df.dropna(subset=["Churn", "NPS"])

# Alvo e faixas
df["churn_flag"] = (df["Churn"].astype(str).str.strip().str.lower() == "yes").astype(int)

#Tipificação do NPS
df["nps_bucket"] = pd.cut(df["NPS"], bins=[-1, 6, 8, 10],
                          labels=["Detrator (0-6)", "Neutro (7-8)", "Promotor (9-10)"])

#Classificação por faixa de relacionamento.                          
df["tenure_bucket"] = pd.cut(df["tenure"], bins=[-1, 2, 6, 12, 24, 48, 120],
                             labels=["0-2m", "3-6m", "7-12m", "13-24m", "25-48m", "49m+"])

#Classificação por faixa de faturamento.
if "MonthlyCharges" in df.columns:
    df["charge_bucket"] = pd.qcut(df["MonthlyCharges"].rank(method="first"), 5,
                                  labels=["Q1", "Q2", "Q3", "Q4", "Q5"])

# Validações simples
assert df["NPS"].between(0, 10).all(), "Existem valores de NPS fora de [0,10]."
assert (df["tenure"] >= 0).all(), "Existem valores de tenure negativos."

# Checagem de missing values
mv = df.isna().sum().sort_values(ascending=False)
print("Missing values por coluna (top 10):")
print(mv.head(10))

df.head()


Deduplicação por cpf: 8248 -> 7065
Missing values por coluna (top 10):
tenure           0
PhoneService     0
tenure_bucket    0
nps_bucket       0
churn_flag       0
Dependents       0
Partner          0
SeniorCitizen    0
gender           0
NPS              0
dtype: int64


,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,cpf,NPS,gender,SeniorCitizen,Partner,Dependents,churn_flag,nps_bucket,tenure_bucket,charge_bucket
0,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),118.75,8672.45,No,10433218100,9,Female,0,Yes,Yes,0,Promotor (9-10),49m+,Q5
1,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Electronic check,118.65,8477.60,No,96001338914,9,Female,0,No,No,0,Promotor (9-10),49m+,Q5
2,68,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Mailed check,118.60,7990.05,No,8386379499,9,Female,0,Yes,No,0,Promotor (9-10),49m+,Q5
3,61,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Electronic check,118.60,7365.70,No,2654235114,8,Female,0,No,No,0,Neutro (7-8),49m+,Q5
4,67,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,Yes,Bank transfer (automatic),118.35,7804.15,Yes,16155940789,9,Male,0,No,No,1,Promotor (9-10),49m+,Q5



## 5) Análise Exploratória

In [0]:

# Métricas gerais de churn/NPS
n_clients = len(df)

churn_rate = df["churn_flag"].mean()

nps_contagem = df["nps_bucket"].value_counts(normalize=True).reindex(
    ["Detrator (0-6)", "Neutro (7-8)", "Promotor (9-10)"]).fillna(0.0)

nps_promotor = float(nps_contagem.get("Promotor (9-10)", 0.0))
nps_detrator = float(nps_contagem.get("Detrator (0-6)", 0.0))
nps_score = (nps_promotor - nps_detrator) * 100

summary = pd.DataFrame({
    "Métrica": ["Clientes", "Churn rate (%)", "NPS (score)",
                "Promotores (%)", "Detratores (%)"],
    "Valor": [n_clients, round(churn_rate*100,2), round(nps_score,1),
              round(nps_promotor,2)*100, round(nps_detrator,2)*100]
})

summary


,Métrica,Valor
0,Clientes,7065.00
1,Churn rate (%),26.55
2,NPS (score),7.70
3,Promotores (%),43.00
4,Detratores (%),35.00


In [0]:
#Criação dos graficos de NPS e Faixa de Faturamento

fig1 = px.histogram(
    df.dropna(subset=["MonthlyCharges"]),
    x="MonthlyCharges",
    nbins=40,
    title="Distribuição das faixas de Faturamento",
    text_auto=True
)
fig1.update_layout(
    xaxis_title="Faturamento",
    yaxis_title="Frequência",
    plot_bgcolor="white",
    paper_bgcolor="white"
)
fig1.update_traces(
    marker_color="#8A2BE2"
)
fig1.show()

fig2 = px.histogram(
    df.dropna(subset=["NPS"]),
    x="NPS",
    nbins=11,
    title="Distribuição de NPS",
    text_auto=True
)
fig2.update_layout(
    xaxis_title="NPS (0-10)",
    yaxis_title="Frequência",
    plot_bgcolor="white",
    paper_bgcolor="white"
)
fig2.update_traces(
    marker_color="#8A2BE2"
)
fig2.show()

In [0]:

# Churn por segmentos
def churn_table(df, col):
    t = (df.groupby(col)["churn_flag"].agg(rate="mean", clientes="count").reset_index())
    t["rate_pct"] = (t["rate"]*100).round(2)
    return t.sort_values("rate", ascending=False)

dims = ["Contract","InternetService","TechSupport","PaymentMethod","tenure_bucket","charge_bucket","PaperlessBilling"]
tables = {d: churn_table(df, d) for d in dims if d in df.columns}
tables["Contract"]


,Contract,rate,clientes,rate_pct
0,Month-to-month,0.426883,3891,42.69
1,One year,0.112162,1480,11.22
2,Two year,0.028926,1694,2.89


In [0]:
for key in [
    "Contract",
    "InternetService",
    "TechSupport",
    "tenure_bucket",
    "charge_bucket",
    "PaymentMethod"
]:
    if key in tables:
        t = tables[key]
        fig = px.bar(
            t,
            x=key,
            y="rate_pct",
            title=f"% de Churn por {key}",
            text_auto=True
        )
        fig.update_layout(
            xaxis_title=key,
            yaxis_title="% de Churn",
            plot_bgcolor="white",
            paper_bgcolor="white"
        )
        fig.update_traces(
            marker_color="#8A2BE2"
        )
        fig.show()


## 6) Próxima Fase
- Aplicação do modelo de Random Forest (Tunning e Matrix de Confusão)
- Criação do modelo Next Best Action.

